In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [3]:
# Đọc dữ liệu từ CSV
df = pd.read_csv("D:\\House Prices _ onehot\\Dataset\\traning_data.csv")
df.describe()

,Current Floor,Total Floor,Bathroom,Balcony,Car Parking,Amount,Area,Amount_Category_Low,Amount_Category_Medium,Amount_Category_High,Types_ Covered,Types_ Open,Types_Unknown,Furnishing_Furnished,Furnishing_Semi-Furnished,Furnishing_Unfurnished,Furnishing_Unknown
count,175547.000000,175547.000000,175547.000000,175547.000000,175547.000000,1.755470e+05,1.755470e+05,175547.000000,175547.000000,175547.000000,175547.000000,175547.000000,175547.000000,175547.000000,175547.000000,175547.000000,175547.000000
mean,4.740856,8.781215,2.456305,2.013979,2.888896,8.704995e+06,2.259135e+03,0.244789,0.504275,0.250936,0.399249,0.063960,0.536791,0.105465,0.470991,0.415877,0.007667
std,4.425971,7.370100,0.848510,0.816974,2.426943,1.831435e+07,2.027501e+05,0.429963,0.499983,0.433553,0.489745,0.244683,0.498646,0.307152,0.499159,0.492874,0.087228
min,1.000000,1.000000,1.000000,1.000000,1.000000,-2.147484e+09,1.000000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,4.000000,2.000000,1.000000,1.000000,5.000000e+05,9.000000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,6.000000,2.000000,2.000000,4.000000,7.800000e+05,1.200000e+03,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,5.000000,11.000000,3.000000,2.000000,4.000000,1.490000e+07,1.600000e+03,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
max,63.000000,70.000000,10.000000,10.000000,50.000000,8.000000e+08,6.534000e+07,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
X = df[['Current Floor', 'Total Floor', 'Bathroom', 'Balcony', 'Car Parking', 'Area', 'Types_ Covered', 'Types_ Open', 'Types_Unknown', 'Furnishing_Furnished', 'Furnishing_Semi-Furnished', 'Furnishing_Unfurnished', 'Furnishing_Unknown']].values  # Đầu vào
y = df[['Amount_Category_Low', 'Amount_Category_Medium', 'Amount_Category_High']].values       # Đầu ra (giá nhà đã log)


In [5]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
print(X_train.shape)  # Kỳ vọng: (n_samples, n_features)
print(y_train.shape)  # Kỳ vọng: (n_samples, 3) nếu one-hot đúng


(140437, 13)
(140437, 3)


In [8]:
torch_X_train = torch.tensor(X_train).type(torch.float32)
torch_y_train = torch.tensor(np.array(y_train)).type(torch.float32)

torch_X_test = torch.tensor(X_test).type(torch.float32)
torch_y_test = torch.tensor(np.array(y_test)).type(torch.float32)

In [26]:
# Định nghĩa lại mô hình phù hợp với dữ liệu mới (4 input features)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(13, 64)  # 4 đặc trưng đầu vào
        self.bn1 = nn.BatchNorm1d(64)
        self.layer2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(32)
        self.layer3 = nn.Linear(32, 16)
        self.bn3 = nn.BatchNorm1d(16)
        self.out = nn.Linear(16, 3)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.relu(self.bn1(self.layer1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.layer2(x)))
        x = self.dropout(x)
        x = self.relu(self.bn3(self.layer3(x)))
        x = self.out(x)
        return x


In [19]:
net = Net()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [20]:
# Tạo DataLoader để huấn luyện theo batch (tăng tốc độ và ổn định)
batch_size = 64
train_data = TensorDataset(torch_X_train, torch_y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [21]:
# Training loop
epochs = 50
for epoch in range(epochs):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = net(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if epoch % 5 == 0:  # In ra loss sau mỗi 5 epochs
        print(f"Epoch {epoch}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 0/50, Loss: 0.1580
Epoch 5/50, Loss: 0.1222
Epoch 10/50, Loss: 0.1187
Epoch 15/50, Loss: 0.1150
Epoch 20/50, Loss: 0.1127
Epoch 25/50, Loss: 0.1105
Epoch 30/50, Loss: 0.1097
Epoch 35/50, Loss: 0.1081
Epoch 40/50, Loss: 0.1070
Epoch 45/50, Loss: 0.1070


In [22]:
with torch.no_grad():
    y_pred = net(torch_X_test)
    predictions = torch.argmax(y_pred, dim=1)

    # Kiểm tra nếu y_test đã ở dạng chỉ số (integer)
    if torch_y_test.dim() > 1:
        # Chuyển y_test từ one-hot encoding về dạng chỉ số
        torch_y_test = torch.argmax(torch_y_test, dim=1)

    accuracy = (predictions == torch_y_test).sum().item() * 100/ torch_y_test.size(0)

print(f"Accuracy: {accuracy:.4f}%")


Accuracy: 70.0256%


In [25]:
# Lưu trọng số của mô hình
torch.save(net.state_dict(), 'D:\\House Prices _ onehot\\src\\model\\house_price_model.pth')
